Experiement Sampling

In [31]:
import pandas as pd
import numpy as np
import faker


Fake case: 

Every customer service agent has a customer rating. To reduce customer complaints and improve rating, the company plans to roll out a X hour training. Before scaling up the program, the company would like to run an experiement to see whether the training helps reduce customer complaints. 

Step 1 Create a dummy sample data set

In [63]:
fake = faker.Faker()

# function to create a dataframe with fake values for our workers
def create_agents(num):
    
    state = [fake.state() for x in range(10)]

    agents = [{'Agent ID': x + 1000,
                  'Agent Name': fake.name(), 
                #   'Agent Tenure': int(np.random.uniform(0, 1000, 1)), 
                #   'State': np.random.choice(state),
                  'Compliant Rate': np.random.rand() * 0.5, # complaint rate between 0 and 50%, uniform dist.
                  'Customer Calls': int(np.random.uniform(0, 2000, 1)) # customer calls received between 0 and 2000, uniform dist.
                  } for x in range(num)
                ]
        
    return agents

df_agent = pd.DataFrame(create_agents(num = 1000))
df_agent.head()

,Agent ID,Agent Name,Compliant Rate,Customer Calls
0,1000,Lawrence Williams,0.463068,460
1,1001,Miguel Joseph,0.358190,361
2,1002,Carol Gill,0.201657,1334
3,1003,Susan Lyons,0.274271,177
4,1004,Stephen Thompson,0.380912,118


,Agent ID,Agent Name,Compliant Rate,Customer Calls
0,1000,Lawrence Williams,0.463068,460
1,1001,Miguel Joseph,0.358190,361
2,1002,Carol Gill,0.201657,1334
3,1003,Susan Lyons,0.274271,177
4,1004,Stephen Thompson,0.380912,118
5,1005,Tina Henderson,0.211038,616
6,1006,James Bass,0.386049,314
7,1007,Robert Pratt,0.076698,166
8,1008,James Scott,0.156969,1820
9,1009,Thomas Coleman,0.090810,533
